In [1]:
import tensorflow as tf
from keras.applications import VGG16
from keras import layers, models
from tqdm import tqdm

import os

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)


# Prepare the dataset using tf.data.Dataset
def prepare_dataset(data_dir, batch_size):
    print("Preparing the dataset...")
    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        image_size=(224, 224),
        batch_size=batch_size,
        label_mode='categorical',
        labels='inferred',
        validation_split=0.2,
        subset='training',
        seed=42
    )

    validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        image_size=(224, 224),
        batch_size=batch_size,
        label_mode='categorical',
        labels='inferred',
        validation_split=0.2,
        subset='validation',
        seed=42
    )

    print("Dataset preparation complete.")
    return train_dataset, validation_dataset

# Release GPU memory when done or interrupted
def release_gpu_memory():
    print("Releasing GPU memory...")
    tf.keras.backend.clear_session()
    print("GPU memory released.")



2023-07-27 17:40:46.028326: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 17:40:47.496202: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1
2.13.0


2023-07-27 17:40:49.449304: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-27 17:40:49.472492: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-27 17:40:49.472976: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
# Specify the paths
data_dir = "Dataset/Rotated Images"
batch_size = 16

# Prepare the dataset
train_dataset, validation_dataset = prepare_dataset(data_dir, batch_size)

Preparing the dataset...
Found 127132 files belonging to 4 classes.
Using 101706 files for training.


2023-07-27 17:41:08.381039: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-27 17:41:08.381373: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-27 17:41:08.381640: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-27 17:41:09.428225: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-27 17:41:09.428552: I tensorflow/compile

Found 127132 files belonging to 4 classes.
Using 25426 files for validation.
Dataset preparation complete.


In [3]:
from keras.layers import BatchNormalization

def build_model(num_classes):
    print("Building the model...")
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze the layers in the base model
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = layers.Flatten()(x)
    x = layers.Dense(4096, activation='relu')(x)
    x = BatchNormalization()(x)  # Add BatchNormalization layer here
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(4096, activation='relu')(x)
    x = BatchNormalization()(x)  # Add BatchNormalization layer here
    x = layers.Dropout(0.7)(x)
    predictions = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=predictions)
    print("Model building complete.")
    return model


# Build the model

# Number of classes (4 rotations: 0, 90, 180, 270)
num_classes = 4

model = build_model(num_classes)


Building the model...


Model building complete.


In [4]:
# Compile and train the model
def train_model(model, train_dataset, validation_dataset, num_epochs):
    try:
        print("Compiling the model...")
        # Custom learning rate (choose your desired learning rate)
        learning_rate = 0.0001

        # Create a custom Adam optimizer with the desired learning rate
        optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        print("Model compilation complete.")

        print("Training the model...")
        for epoch in tqdm(range(num_epochs), desc="Epochs"):
            model.fit(train_dataset, validation_data=validation_dataset, epochs=1)
        print("Model training complete.")

    except KeyboardInterrupt:
        print("Training interrupted.")
    finally:
        # Release GPU memory when done or interrupted
        release_gpu_memory()


In [5]:
# Training parameters
num_epochs = 10

# Train the model
train_model(model, train_dataset, validation_dataset, num_epochs)

Compiling the model...
Model compilation complete.
Training the model...


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

2023-07-27 17:41:32.966749: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2023-07-27 17:41:34.250186: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2023-07-27 17:41:35.497946: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-07-27 17:41:36.000873: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2

   1/6357 [..............................] - ETA: 12:02:21 - loss: 2.1653 - accuracy: 0.4375

2023-07-27 17:41:36.846203: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.22GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


...

6357/6357 [==============================] - 598s 94ms/step - loss: 0.0910 - accuracy: 0.9663 - val_loss: 0.1938 - val_accuracy: 0.9403


Epochs:  90%|█████████ | 9/10 [1:29:35<09:57, 597.37s/it]

6357/6357 [==============================] - 600s 94ms/step - loss: 0.0820 - accuracy: 0.9701 - val_loss: 0.1859 - val_accuracy: 0.9454


Epochs: 100%|██████████| 10/10 [1:39:35<00:00, 597.58s/it]

Model training complete.
Releasing GPU memory...
GPU memory released.


In [6]:
from keras.models import save_model

def save_trained_model(model, save_path):
    save_model(model, save_path)
    print("Model saved to:", save_path)


save_path = "Model/"  # Replace with your desired path
save_trained_model(model, save_path)

INFO:tensorflow:Assets written to: Model/assets


INFO:tensorflow:Assets written to: Model/assets


Model saved to: Model/
